In [1]:
import pandas as pd
import numpy as np
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import time
from collections import Counter
from dataclasses import dataclass
from timeit import default_timer as timer
import random
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import gensim
from gensim.models import Word2Vec

import spacy
from spacy import displacy

from spacy.matcher import Matcher
from spacy.tokens import Span

import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


from IPython.display import display, HTML

from typing import List, Callable, Dict, Tuple, Set

pd.set_option('max_colwidth', 600)
pd.set_option('display.max_rows', 500)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=28d5a7ef45349be02dd7945da4c79c6d11e6252c73fa743ae010dd3012e15e77
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
#Load Sentence Transformer model optimized for  sentence cosine similarity calculations

#The models below fully downloaded in Google Colab. This is the version of the google colab notebook but
#it was open in anoconda to be saved as pdf and the download graphics did not transfer properly so
#it seems like it didn't download. However, it did in the orignal google colab notebook, where all the
#analysis was run.

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Only run this once, they will be downloaded.
nltk.download('stopwords',quiet=True)
nltk.download('wordnet',quiet=True)
nltk.download('punkt',quiet=True)
nltk.download('omw-1.4',quiet=True)

True

In [5]:
CORPUS_PATH=\
'gdrive/MyDrive/Colab Notebooks/MSDS453/FinalProject/parks_clean.csv'
national_parks_corpus = pd.read_csv(CORPUS_PATH)
national_parks_corpus.head()

,park,text,char_length,token_lengths
0,Acadia National Park,"Acadia National Park is an American national park located along the mid-section of the Maine coast, southwest of Bar Harbor. The park preserves about half of Mount Desert Island, part of the Isle au Haut, the tip of the Schoodic Peninsula, and portions of 16 smaller outlying islands. It protects the natural beauty of the rocky headlands, including the highest mountains along the Atlantic coast. Acadia boasts a glaciated coastal and island landscape, an abundance of habitats, a high level of biodiversity, clean air and water, and a rich cultural heritage.The park contains the tallest mounta...",46957,7341
1,Arches National Park,"Arches National Park is a national park in eastern Utah, United States. The park is adjacent to the Colorado River, 4 mi (6 km) north of Moab, Utah. More than 2,000 natural sandstone arches are located in the park, including the well-known Delicate Arch, as well as a variety of unique geological resources and formations. The park contains the highest density of natural arches in the world.The park consists of 310.31 km2 (76,680 acres; 119.81 sq mi; 31,031 ha) of high desert located on the Colorado Plateau. The highest elevation in the park is 5,653 ft (1,723 m) at Elephant Butte, and the l...",16175,2536
2,Badlands National Park,"Badlands National Park (Lakota: Makȟóšiča) is an American national park located in southwestern South Dakota. The park protects 242,756 acres (379.3 sq mi; 982.4 km2) of sharply eroded buttes and pinnacles, along with the largest undisturbed mixed grass prairie in the United States. The National Park Service manages the park, with the South Unit being co-managed with the Oglala Lakota tribe.The Badlands Wilderness protects 64,144 acres (100.2 sq mi; 259.6 km2) of the park's North Unit as a designated wilderness area, and is one site where the black-footed ferret, one of the most endangered...",19424,3049
3,Big Bend National Park,"Big Bend National Park is an American national park located in West Texas, bordering Mexico. The park has national significance as the largest protected area of Chihuahuan Desert topography and ecology in the United States, and was named after a large bend in the Rio Grande/Río Bravo. The park protects more than 1,200 species of plants, more than 450 species of birds, 56 species of reptiles, and 75 species of mammals. Additional park activities include scenic drives, programs led by Big Bend park rangers, and stargazing.The area has a rich cultural history, from archeological sites dating ...",23315,3722
4,Biscayne National Park,"Biscayne National Park is an American national park located south of Miami, Florida in Miami-Dade County. The park preserves Biscayne Bay and its offshore barrier reefs. Ninety-five percent of the park is water, and the shore of the bay is the location of an extensive mangrove forest. The park covers 172,971 acres (270.3 sq mi; 700.0 km2) and includes Elliott Key, the park's largest island and northernmost of the true Florida Keys, formed from fossilized coral reef. The islands farther north in the park are transitional islands of coral and sand. The offshore portion of the park includes t...",49014,7817


In [6]:
national_parks_corpus['lower_text'] = national_parks_corpus['text'].str.lower()
national_parks_corpus['sent_tokens'] = national_parks_corpus['lower_text'].apply(nltk.sent_tokenize)
national_parks_corpus['word_tokens'] = national_parks_corpus['lower_text'].apply(nltk.word_tokenize)
national_parks_corpus.head()

,park,text,char_length,token_lengths,lower_text,sent_tokens,word_tokens
0,Acadia National Park,"Acadia National Park is an American national park located along the mid-section of the Maine coast, southwest of Bar Harbor. The park preserves about half of Mount Desert Island, part of the Isle au Haut, the tip of the Schoodic Peninsula, and portions of 16 smaller outlying islands. It protects the natural beauty of the rocky headlands, including the highest mountains along the Atlantic coast. Acadia boasts a glaciated coastal and island landscape, an abundance of habitats, a high level of biodiversity, clean air and water, and a rich cultural heritage.The park contains the tallest mounta...",46957,7341,"acadia national park is an american national park located along the mid-section of the maine coast, southwest of bar harbor. the park preserves about half of mount desert island, part of the isle au haut, the tip of the schoodic peninsula, and portions of 16 smaller outlying islands. it protects the natural beauty of the rocky headlands, including the highest mountains along the atlantic coast. acadia boasts a glaciated coastal and island landscape, an abundance of habitats, a high level of biodiversity, clean air and water, and a rich cultural heritage.the park contains the tallest mounta...","[acadia national park is an american national park located along the mid-section of the maine coast, southwest of bar harbor., the park preserves about half of mount desert island, part of the isle au haut, the tip of the schoodic peninsula, and portions of 16 smaller outlying islands., it protects the natural beauty of the rocky headlands, including the highest mountains along the atlantic coast., acadia boasts a glaciated coastal and island landscape, an abundance of habitats, a high level of biodiversity, clean air and water, and a rich cultural heritage.the park contains the tallest mo...","[acadia, national, park, is, an, american, national, park, located, along, the, mid-section, of, the, maine, coast, ,, southwest, of, bar, harbor, ., the, park, preserves, about, half, of, mount, desert, island, ,, part, of, the, isle, au, haut, ,, the, tip, of, the, schoodic, peninsula, ,, and, portions, of, 16, smaller, outlying, islands, ., it, protects, the, natural, beauty, of, the, rocky, headlands, ,, including, the, highest, mountains, along, the, atlantic, coast, ., acadia, boasts, a, glaciated, coastal, and, island, landscape, ,, an, abundance, of, habitats, ,, a, high, level, of..."
1,Arches National Park,"Arches National Park is a national park in eastern Utah, United States. The park is adjacent to the Colorado River, 4 mi (6 km) north of Moab, Utah. More than 2,000 natural sandstone arches are located in the park, including the well-known Delicate Arch, as well as a variety of unique geological resources and formations. The park contains the highest density of natural arches in the world.The park consists of 310.31 km2 (76,680 acres; 119.81 sq mi; 31,031 ha) of high desert located on the Colorado Plateau. The highest elevation in the park is 5,653 ft (1,723 m) at Elephant Butte, and the l...",16175,2536,"arches national park is a national park in eastern utah, united states. the park is adjacent to the colorado river, 4 mi (6 km) north of moab, utah. more than 2,000 natural sandstone arches are located in the park, including the well-known delicate arch, as well as a variety of unique geological resources and formations. the park contains the highest density of natural arches in the world.the park consists of 310.31 km2 (76,680 acres; 119.81 sq mi; 31,031 ha) of high desert located on the colorado plateau. the highest elevation in the park is 5,653 ft (1,723 m) at elephant butte, and the l...","[arches national park is a national park in eastern utah, united states., the park is adjacent to the colorado river, 4 mi (6 km) north of moab, utah., more than 2,000 natural sandstone arches are located in the park, including the well-known deli

In [7]:
sent_tokens = [item for sublist in national_parks_corpus['sent_tokens'].tolist() for item in sublist]
word_tokens = [item for sublist in national_parks_corpus['word_tokens'].tolist() for item in sublist]

In [8]:
#create greetings and greetings function

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["Hello"]


# Checking for greetings
def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [9]:

# Generating response function
def response(user_response):
    chatbot_response=''
    sentence_encodings=model.encode(sent_tokens, convert_to_tensor=True)# generate sentence transformer embeddings
    sentence_encodings=sentence_encodings.cpu()
    vals = cosine_similarity(sentence_encodings[-1].reshape(1, -1), sentence_encodings) #the chatbot conversation code
    #in the next cell adds the question as the last sentence of the sentence tokens, before calling this response function.
    #The code takes the last sentence (which is the question) and gets cosine similarities vs all the sentences in the corpus,
    #including itself
    idx=vals.argsort()[0][-2] #gets the index of the second highest similarity (the first highest would be the question itself)
    flat = vals.flatten()#reduces dimension of cosine similarity array to be able to sort
    flat.sort() #sort the cosine similarity values
    second_cos_sim_val = flat[-2] #get the second highest cosine similarity value.
    if(second_cos_sim_val==0): #check the second highest cosine similarity value. If it's zero return the no match response,
        #else return highest cosine similarity sentence.
        chatbot_response=chatbot_response+"Sorry, I do not have an answer to your question in my database"
        return chatbot_response
    else:
        chatbot_response = chatbot_response+sent_tokens[idx] #use index of highest cosine similarity to get original sentence
        return chatbot_response

In [10]:
#Chatbot interaction code

flag=True
print("Welcome to the US National Parks Information Chatbot. To end session please type exit")
print("\n")
times = []

while(flag==True):
    user_response = input()
    start = time.time()
    user_response=user_response.lower()
    if user_response!='exit':
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Answer: You are welcome!")
        else:
            if(greeting(user_response)!=None):
                print("Answer: "+greeting(user_response))
            else:
                sent_tokens.append(user_response)
                word_tokens=word_tokens+nltk.word_tokenize(user_response)
                final_words=list(set(word_tokens))
                print("Answer: ",end="")
                print(response(user_response))
                end = time.time()
                print("\n")
                times.append(end-start)
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Thank you for using the US National Parks Information Chatbot. Good bye.")
        print(f"\nAverage response time: {round(np.mean(times),2)} seconds")


Welcome to the US National Parks Information Chatbot. To end session please type exit


When was Yellowstone National Park established?
Answer: it was established by the 42nd u.s. congress with the yellowstone national park protection act and signed into law by president ulysses s. grant on march 1, 1872. yellowstone was the first national park in the u.s. and is also widely held to be the first national park in the world.


What factors make the Grand Canyon unique?
Answer: the national parks: america's best idea: parks-grand canyon.


What activities are available in Shenandoah National Park?
Answer: it is also designated as a national scenic byway.shenandoah national park offers 196,000 acres (306.2 sq mi; 793.2 km2) of backcountry and wilderness camping.


Which National Parks are located in California?
Answer: the national parks.


Explain the potential effects of climate change on Glacier National Park's glaciers and ecosystems. How can the park adapt to these changes?
Answer: th